In [ ]:
%matplotlib inline


# Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation


This is an example of applying :class:`sklearn.decomposition.NMF` and
:class:`sklearn.decomposition.LatentDirichletAllocation` on a corpus
of documents and extract additive models of the topic structure of the
corpus.  The output is a list of topics, each represented as a list of
terms (weights are not shown).

Non-negative Matrix Factorization is applied with two different objective
functions: the Frobenius norm, and the generalized Kullback-Leibler divergence.
The latter is equivalent to Probabilistic Latent Semantic Indexing.

The default parameters (n_samples / n_features / n_components) should make
the example runnable in a couple of tens of seconds. You can try to
increase the dimensions of the problem, but be aware that the time
complexity is polynomial in NMF. In LDA, the time complexity is
proportional to (n_samples * iterations).




In [6]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [7]:
print("Loading dataset")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))

Loading dataset


In [12]:
import pandas as pd
dataset = pd.read_pickle('data_clean.pkl')

In [13]:
pd.set_option('max_colwidth', 1500)
dataset

,text
0,thank you so much chris and it is truly a great honor to have the opportunity to come to this stage twice I am extremely grateful i have been blown away by this conference and i want to thank all of you for the many nice comment about what i had to say the other night and i say that sincerely partly because mock sob i need that laughter put yourselves in my position laughter i flew on air force two for eight year laughter now i have to take off my shoe or boot to get on an airplane laughter applause I will tell you one quick story to illustrate what that is been like for me laughter it is a true story every bit of this is true soon after tipper and i left the mock sob white house laughter we were driving from our home in nashville to a little farm we have mile east of nashville driving ourselves laughter i know it sound like a little thing to you but laughter i looked in the rearview mirror and all of a sudden it just hit me there wa no motorcade back there laughter you have heard of phantom limb pain laughter this wa a rented ford taurus laughter it wa dinnertime and we started looking for a place to eat we were on we got to exit lebanon tennessee we got off the exit we found a shoneys restaurant lowcost family restaurant chain for those of you who do not know it we went in and sat down at the booth and the waitress came over made a big commotion over tipper laughter she took our order...
9,if you are here today and I am very happy that you are you have all heard about how sustainable development will save u from ourselves however when we are not at ted we are often told that a real sustainability policy agenda is just not feasible especially in large urban area like new york city and that is because most people with decisionmaking power in both the public and the private sector really do not feel a though they are in danger the reason why I am here today in part is because of a dog an abandoned puppy i found back in the rain back in she turned out to be a much bigger dog than I would anticipated when she came into my life we were fighting against a huge waste facility planned for the east river waterfront despite the fact that our small part of new york city already handled more than percent of the entire city commercial waste a sewage treatment pelletizing plant a sewage sludge plant four power plant the world largest fooddistribution center a well a other industry that bring more than diesel truck trip to the area each week the area also ha one of the lowest ratio of park to people in the city so when i wa contacted by the park department about a seedgrant initiative to help develop waterfront project i thought they were really wellmeaning but a bit naive I would lived in this area all my life and you could not get to the river because of all the lovely facility that i mentioned earlier then while jogging with my dog one morning she pulled ...
18,good morning how are you laughter it is been great ha not it I have been blown away by the whole thing in fact I am leaving laughter there have been three theme running through the conference which are relevant to what i want to talk about one is the extraordinary evidence of human creativity in all of the presentation that we have had and in all of the people here just the variety of it and the range of it the second is that it is put u in a place where we have no idea what is going to happen in term of the future no idea how this may play out i have an interest in education actually what i find is everybody ha an interest in education do not you i find this very interesting if you are at a dinner party and you say you work in education actually you are not often at dinner party frankly laughter if you work in education you are not asked laughter and you are never asked back curiously that is strange to me but if you are and you say to somebody you know they say what do you do and you say you work in education you can see the blood run from their face they are like oh my 

In [14]:
n_samples = 1591
n_features = 1000
n_components = 10
n_top_words = 20

# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

data_samples = dataset.text[:n_samples]
print("done in %0.3fs." % (time() - t0))

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

done in 108.475s.
Extracting tf-idf features for NMF...
done in 2.402s.
Extracting tf features for LDA...
done in 2.469s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=1591 and n_features=1000...
done in 1.777s.

Topics in NMF model (Frobenius norm):
Topic #0: said laughter people know life did story time child say day just love year got man thing school kid family
Topic #1: woman men girl sex female boy violence male mother village daughter young black man mom talk role country job community
Topic #2: people country world government percent year global dollar need ha problem state change company social economy china democracy money economic
Topic #3: cancer cell patient disease drug body doctor tissue health blood treatment medical medicine dna gene protein trial virus heart bone
Topic #4: planet earth universe water star specie ocean year ice animal plant life energy light solar tree climate sun mar space
Topic #5: city building design architecture space com